In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys

sys.path.insert(1, '/mnt/c/scripts/tools/')

from bigquery_bucket_tools import *
from plot_tools import *
from tools import *

from datetime import datetime, timedelta, date 
from dateutil.relativedelta import *
from netCDF4 import Dataset   
import glob
import requests 
import pandas as pd
import xarray as xr
import requests 
import functools
import operator
import wget
import os
import gc
import numpy as np
import pygrib

import geopandas as gpd
import shapely
import shapefile
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import cartopy, cartopy.crs as ccrs   
import geopandas as gpd
import regionmask
from shapely.geometry import Point, Polygon

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker 
import scipy.ndimage as ndimage
from PIL import Image
from scipy.ndimage import gaussian_filter
import scipy.ndimage

from google.cloud import storage, bigquery

In [ ]:
#Data de referência 

datetime.today() - relativedelta(month=1)
data = datetime.strptime('2023-03-31', '%Y-%m-%d') 
data_ = data.date().strftime('%Y-%m-01') 


# selecionar a data do mês de feva partir do dia 
data_ini = (data - relativedelta(months=1)).date().strftime('%Y-%m-01') 


##Extrai o ultimo dia do mes
data_fim = (datetime.strptime(data_,'%Y-%m-%d') - timedelta(days=datetime.strptime(data_,'%Y-%m-%d').day)).date().strftime('%Y-%m-%d')

**Consultas BigQuery**

Merge

In [ ]:
#Consulta merge_precipitacao

root='/mnt/c/scripts/relatorio_mensal/resultados/'

direc='/mnt/c/scripts/dados_plot'


project_name = 'modelagem-de-precos'
dataset ='cptec'
table ='merge_precipitacao'
ref = 'date'

client = storage.Client(project=project_name)
bq_client = bigquery.Client(project=project_name)

#modelagem-de-precos.cptec.merge_precipitacao

# MERGE DAILY 
merge = bq_client.query(
f'''SELECT * FROM `{project_name}.{dataset}.{table}` WHERE DATE({ref}) BETWEEN "{data_ini}" AND "{data_fim}" ORDER BY DATE({ref})''').result().to_dataframe().drop_duplicates(keep='first', ignore_index=True)

*Modelo EC- Sazonal*


In [ ]:
#Consulta EC- Sazonal 


dataset ='copernicus_ecmwf'
table ='precipitation_ecens7m'
ref = 'forecast'


#1 - filtrar com a data
#2-  fazer o esemble (agrupa por tudo menos o mes)

ec = bq_client.query(
f'''WITH table1 AS (
SELECT  
  mes_ref
, forecast
, membro
, latitude
, longitude
, prec_diaria 
FROM `{project_name}.{dataset}.{table}` 


WHERE DATE(forecast) 

BETWEEN "{data_ini}" AND "{data_fim}" 

ORDER BY DATE({ref}))

SELECT 
  mes_ref
, forecast
, latitude
, longitude
, avg(prec_diaria) as precipitation

FROM table1 

GROUP BY mes_ref , forecast , latitude, longitude

''').result().to_dataframe().drop_duplicates(keep='first', ignore_index=True) #retira indices iguais 




*Climatologia*
----------------------------------------
Os dados não possuem ano, é apenas uma forma de organização no google cloud


#Consulta CLIMATOLOGIA 


def read_query_nested(list_month: list, project: str, dataset: str, table: str, nested_flag: str ) -> pd.DataFrame:
    
    bq_client = bigquery.Client(project=project)

    query_bq = "SELECT monthofyear, longitude, nested.* FROM {}, UNNEST({}) AS {} WHERE extract(month from monthofyear) in ({}) ".format(f"{project}.{dataset}.{table}", nested_flag, nested_flag, list_months)

    dataframe = bq_client.query(query_bq).result().to_dataframe()

    dataframe.drop_duplicates(keep='first', inplace=True, ignore_index=True)
            
    return dataframe


In [ ]:
list_months = str([int((data - relativedelta(months=month)).strftime('%m')) for month in np.arange(1,2)]).replace('[', '').replace(']', '')
print(list_months) #mês de fevereiro 



climatologia = read_query_nested(list_months, 'modelagem-de-precos', 'climatologies', 'merge_monthly_accumulated_grid', 'nested')

climatologia = climatologia.rename({'monthofyear': 'forecast_month'}, axis=1)

climatologia['forecast_month'] = pd.to_datetime(climatologia['forecast_month']).dt.strftime('%m')

climatologia = climatologia[['latitude', 'longitude', 'forecast_month', 'precipitation']]





def manipula_frame(dataframe: pd.DataFrame, string_forecast) -> pd.DataFrame:

    lista = sorted(dataframe[string_forecast].unique())
      
    dataframe = dataframe.groupby([string_forecast, 'lat', 'lon']).sum().reset_index() 
    
    dataframe.sort_values(by=[string_forecast, 'lat', 'lon'], ascending=True, inplace=True)

    dataframe.set_index(['lat', 'lon']).to_xarray()
    
    return dataframe

